In [1]:
import numpy as np
import matplotlib.pyplot as plt
from finite_volume.advection import AdvectionSolver
from finite_volume.initial_conditions import generate_ic


def l1(x: np.ndarray) -> float:
    return np.mean(np.abs(x))


def l2(x: np.ndarray) -> float:
    return np.sqrt(np.mean(np.square(x)))


def linf(x: np.ndarray) -> float:
    return np.max(np.abs(x))

In [2]:
p = 2
a = 1
k = -2
quadrature = "gauss-legendre"
ic_type__PAD = "square", (0, 1)
limiter_config = dict(
    aposteriori_limiting=True,
    SED=False,
    hancock=False,
    convex=False,
    fallback_to_first_order=False,
    fallback_limiter="minmod",
)

In [3]:
ic_type, PAD = ic_type__PAD


def linear_transformation(x):
    return a * x + k


def u0(x, y):
    return generate_ic(type=ic_type, x=x, y=y)


def u0_shifted(x, y):
    return linear_transformation(u0(x, y))


u0_shifted.__name__ += f"_{a}_{k}"

shared_config = dict(
    **limiter_config,
    save=False,
    load=False,
    v=(2, 1),
    n=(64,),
    order=p + 1,
    courant=0.8,
    snapshot_dt=0.01,
    num_snapshots=100,
    flux_strategy=quadrature,
)

# baseline
solver = AdvectionSolver(
    **shared_config,
    u0=u0,
    PAD=PAD,
)
# solver.one_euler_step()
solver.rkorder()

# shifted initial condition
translated_solver = AdvectionSolver(
    **shared_config,
    u0=u0_shifted,
    PAD=sorted((linear_transformation(PAD[0]), linear_transformation(PAD[1]))),
)
# translated_solver.one_euler_step()
translated_solver.rkorder()

# check equivariance
diffs = (
    linear_transformation(solver.u_snapshots[-1][1])
    - translated_solver.u_snapshots[-1][1]
)
print(f"{l1(diffs)=}")
print(f"{l2(diffs)=}")
print(f"{linf(diffs)=}")

New solution instance...


  0%|          | 0/1.0 [00:00]

100%|█████████▉| 0.998333333333334/1.0 [00:04]   /usr/local/Caskroom/miniconda/base/envs/fv-env/lib/python3.11/site-packages/tqdm/std.py:639: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 1.0000000000000007/1.0 [00:04]



New solution instance...


100%|██████████| 1.0000000000000007/1.0 [00:02]  



l1(diffs)=8.090698516371336e-05
l2(diffs)=0.00027265587069749876
linf(diffs)=0.0064542059307486355
